In [1]:
#not show feature warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import pandas as pd
import tensorflow as tf
import numpy

from scikeras.wrappers import KerasClassifier

from sklearn.metrics import accuracy_score, classification_report, recall_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline

from wittgenstein import RIPPER
from wittgenstein.interpret import interpret_model, score_fidelity


In [2]:
src = "dataset_modified.csv"
data = pd.read_csv(src)
le = LabelEncoder()
for col in data.select_dtypes(include='object'):
    data[col] = le.fit_transform(data[col])
# View updated dataframe
X = data.drop(['Disease'], axis=1)
y = data["Disease"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

x_train = x_train.to_numpy(dtype='float32')
x_test = x_test.to_numpy(dtype='float32')

# Convert target data to numpy arrays with int64 data type
y_train = y_train.to_numpy(dtype='int64')
y_test = y_test.to_numpy(dtype='int64')

In [3]:
num_classes = y.nunique()
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=32, input_shape=(x_train.shape[1], 1), return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(units=16, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=num_classes, activation='softmax')
])
rnn_model = KerasClassifier(build_fn=model, epochs=100, batch_size=32, loss=tf.keras.losses.CategoricalCrossentropy())
# model.compile(optimizer='Nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))


In [4]:
# adaboost_model = make_pipeline(OneHotEncoder(sparse=False, handle_unknown='use_encoded_value'), AdaBoostClassifier(learning_rate=.1))
adaboost_model = AdaBoostClassifier()
gradientboost_model = GradientBoostingClassifier(criterion='squared_error')

In [5]:
ensemble_clf = StackingClassifier(estimators=[('adaboost', adaboost_model), ('rnn', rnn_model), ('gradientboost', gradientboost_model)], final_estimator=LogisticRegression())
ensemble_clf.fit(x_train, y_train)

Epoch 1/100
116/116 [==============================] - 8s 47ms/step - loss: 3.7157
Epoch 2/100
116/116 [==============================] - 5s 43ms/step - loss: 3.7139
Epoch 3/100
116/116 [==============================] - 6s 55ms/step - loss: 3.7122
Epoch 4/100
116/116 [==============================] - 9s 77ms/step - loss: 3.7085
Epoch 5/100
116/116 [==============================] - 9s 77ms/step - loss: 3.5478
Epoch 6/100
116/116 [==============================] - 9s 80ms/step - loss: 3.2043
Epoch 7/100
116/116 [==============================] - 9s 80ms/step - loss: 2.9657
Epoch 8/100
116/116 [==============================] - 10s 84ms/step - loss: 2.7479
Epoch 9/100
116/116 [==============================] - 10s 83ms/step - loss: 2.5959
Epoch 10/100
116/116 [==============================] - 10s 85ms/step - loss: 2.4564
Epoch 11/100
116/116 [==============================] - 10s 85ms/step - loss: 2.3420
Epoch 12/100
116/116 [==============================] - 10s 83ms/step - loss: 2.2

StackingClassifier(estimators=[('adaboost', AdaBoostClassifier()),
                               ('rnn',
                                KerasClassifier(batch_size=32, build_fn=<keras.engine.sequential.Sequential object at 0x000002739F071A10>, epochs=100, loss=<keras.losses.CategoricalCrossentropy object at 0x000002739ED025D0>)),
                               ('gradientboost',
                                GradientBoostingClassifier(criterion='squared_error'))],
                   final_estimator=LogisticRegression())

In [6]:
y_pred = ensemble_clf.predict(x_test)
print("Classification report: ")
print(classification_report(y_test, y_pred))

print("Accuracy: ")
print(accuracy_score(y_test, y_pred))

print("Recall: ")
print(recall_score(y_test, y_pred, average=None))

39/39 [==============================] - 1s 14ms/step
Classification report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        36
           2       1.00      1.00      1.00        33
           3       1.00      1.00      1.00        29
           4       1.00      1.00      1.00        29
           5       1.00      1.00      1.00        30
           6       1.00      1.00      1.00        38
           7       1.00      1.00      1.00        29
           8       1.00      1.00      1.00        30
           9       1.00      1.00      1.00        21
          10       1.00      1.00      1.00        29
          11       1.00      1.00      1.00        29
          12       1.00      1.00      1.00        29
          13       1.00      1.00      1.00        37
          14       1.00      1.00      1.00        30
          15       1.00      1.00      1.00        23
   